# Chunking Strategies III: LLM-based chunking

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from pathlib import Path
from dotenv import load_dotenv
import tiktoken
from langchain_core.documents import Document
from IPython.display import display, Markdown
from openai import OpenAI
from src import utils

# Params

In [3]:
TOKENIZER_NAME ="cl100k_base"
EMB_MODEL="text-embedding-3-small"
LLM = "gpt-4.1-mini-2025-04-14"  # 1M token context, better than gpt-4o in structured output

[Using GPT-5](https://platform.openai.com/docs/guides/latest-model)

gpt5-main-mini no está disponible aun en API

In [4]:
path_data = Path() / "data"
path_input = path_data / "interim"
path_output = path_data / "processed" / "chunks"  # Store chunks by experiment

path_output.mkdir(exist_ok=True, parents=True)

# Environment Variables

In [5]:
load_dotenv()

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [6]:
encoding = tiktoken.get_encoding(TOKENIZER_NAME)

# Clients

In [7]:
client_openai= OpenAI(api_key=OPENAI_API_KEY)

# Load Data

In [8]:

dc_chunks = utils.load_documents(path_output / "pages" / "Althera.json")

corpus_doc = Document(
    page_content="\n\n".join([doc.page_content for doc in dc_chunks[:4]]),
    metadata={"source": "Althera"}
)

In [9]:
stats_doc = utils.corpus_stats(encoding, [corpus_doc])
stats_doc.describe()

count       1.0
mean     2433.0
std         NaN
min      2433.0
25%      2433.0
50%      2433.0
75%      2433.0
max      2433.0
dtype: float64

In [10]:
corpus_doc.page_content[-100:]

'e aproximadamente $\\mathbf{4 . 9 0 0}$ millones de años, una edad comparable a la del Sistema Solar.'

# LLM Chunking (DIY)

Input to an LLM a text based document

In [11]:
from typing import  List


from enum import Enum
from typing import List
from pydantic import BaseModel, Field, constr


class ChunkReason(str, Enum):
    section = "section"
    semantic = "semantic"
    other = "other"


class Chunks(BaseModel):
    text: constr(strip_whitespace=True, min_length=1) = Field(
        ..., description="Exactly the chunked text from the document"
    )
    summary: constr(strip_whitespace=True, min_length=1) = Field(
        ..., description="A concise summary of the extracted text"
    )
    reason_of_chunk: ChunkReason = Field(
        ..., description="The reason why this chunk was extracted."
    )
    sections_found: List[str] = Field(
        default_factory=list,
        description="List of section headers found in this chunk. Can be an list empty.",
    )

class DocumentChunked(BaseModel):
    chunks: List[Chunks] = Field(..., description="List of chunks extracted")


response = client_openai.responses.parse(
    model=LLM,
    input=[
        {"role": "system", "content": "Your task is to split a markdown document text information into chunks.\
         You should follow these roules:\
         Each chunk should be splitted base on natural text separators, such as sentences or paragraphs boundaries.\
         Each chunk ideally should have a self-contained topic.\
         Markdown headers can be missleading, so infer if they are properly placed and use them to split only if they are convenient\
         Aim at a minimum chunk size of 128 tokens and a maximun chunk size of 1024.\
         Of course, you MUST REPRODUCE EXACTLY the same text seen in the input, do not change nothing.\
         "},
        {
            "role": "user",
            "content": f"Text in markdown to chunk: \n\n{corpus_doc.page_content}",
        },
    ],
    text_format=DocumentChunked,
)

chunks = response.output_parsed



In [12]:
chunks = response.output_parsed

len(chunks.chunks)

11

In [13]:
Markdown(chunks.chunks[0].text)

# Un nuevo y fascinante vecino: Althéra 

Índice

1. Historia del descubrimiento
2. Conoce a Althéra
3. Los soles de Althéra
4. Estructura general de Althéra
5. Planetas interiores
6. Planetas exteriores
7. Lunas y satélites menores
8. Fenómenos destacados
9. Habitabilidad y astrobiología
10. Conclusiones y perspectivas futuras

In [14]:
Markdown(chunks.chunks[-1].text)

### 3.3 Ciclo de vida y edad estimada del sistema binario

Estudios de metalicidad realizados con el ELT indican que ambas estrellas tienen composiciones químicas similares, con una proporción de elementos pesados algo mayor que la del Sol ( $[\mathrm{Fe} / \mathrm{H}] \approx+0,08$ ). Esto sugiere que se formaron a partir de la misma nube molecular hace aproximadamente $\mathbf{4 . 9 0 0}$ millones de años, una edad comparable a la del Sistema Solar.

In [15]:
for i, chunk in enumerate(chunks.chunks):
    print(f"chunk: {i} ----------------")
    print(f"{chunk.reason_of_chunk=}")
    print(f"{chunk.summary=}")
    print(f"{chunk.sections_found=}")
    print("Text:")
    display(Markdown(chunk.text))

chunk: 0 ----------------
chunk.reason_of_chunk=<ChunkReason.section: 'section'>
chunk.summary='Introducción al sistema Althéra junto con el índice temático que estructura el contenido del documento.'
chunk.sections_found=['Un nuevo y fascinante vecino: Althéra']
Text:


# Un nuevo y fascinante vecino: Althéra 

Índice

1. Historia del descubrimiento
2. Conoce a Althéra
3. Los soles de Althéra
4. Estructura general de Althéra
5. Planetas interiores
6. Planetas exteriores
7. Lunas y satélites menores
8. Fenómenos destacados
9. Habitabilidad y astrobiología
10. Conclusiones y perspectivas futuras

chunk: 1 ----------------
chunk.reason_of_chunk=<ChunkReason.section: 'section'>
chunk.summary='El descubrimiento inicial del sistema binario Althéra se realizó en 2032 gracias al Observatorio Espacial James Webb, identificando oscilaciones lumínicas que sugerían estrellas en órbita mutua con cuerpos circumbinarios.'
chunk.sections_found=['1. Historia del descubrimiento', '1.1 Primeras observaciones y sospechas iniciales']
Text:


## 1. Historia del descubrimiento

### 1.1 Primeras observaciones y sospechas iniciales

El sistema binario Althéra (HD 4579 AB ) fue detectado por primera vez en el año 2032 durante una campaña de observación del Observatorio Espacial James Webb, dirigida por la astrofísica chilena Dra. Mariela Estay. La misión principal era estudiar la composición atmosférica de exoplanetas candidatos a la habitabilidad, pero un patrón anómalo en el flujo luminoso proveniente de la constelación de Orión llamó la atención del equipo. El análisis de curvas de luz reveló oscilaciones periódicas dobles, un indicio claro de la presencia de dos estrellas en órbita mutua y varios cuerpos orbitando de forma circumbinaria.

chunk: 2 ----------------
chunk.reason_of_chunk=<ChunkReason.semantic: 'semantic'>
chunk.summary='Confirmación del sistema Althéra y sus características orbitales mediante observaciones avanzadas con telescopios ELT, Roman y la red VLBA, detectando cinco planetas principales y restos circunestelares.'
chunk.sections_found=['1.2 Confirmación mediante técnicas combinadas']
Text:


...

## 3. Los soles de Althéra

### 3.1 Clasificación espectral y diferencias principales

El corazón del sistema Althéra está formado por un par estelar bien contrastado:

- Althéra A (HD 4579 A): estrella de tipo F7V, más caliente y luminosa que el Sol, con un tono blanco-dorado.
- Althéra B (HD 4579 B): estrella de tipo K2V, más fría y longeva, de tonalidad naranja intensa.

Esta combinación de espectros produce un ambiente luminoso único en los planetas circumbinarios, donde la radiación es más equilibrada en el rango visible y cercana al infrarrojo, favoreciendo la fotosíntesis de potenciales formas de vida adaptadas a ambas fuentes.

chunk: 9 ----------------
chunk.reason_of_chunk=<ChunkReason.section: 'section'>
chunk.summary='Comparación detallada de masas, radios, temperaturas y luminosidades de Althéra A y B, mostrando el rol estabilizador térmico de la estrella más fría aunque menos luminosa.'
chunk.sections_found=['3.2 Masas, radios y luminosidades comparadas']
Text:


# 3.2 Masas, radios y luminosidades comparadas 

El sistema binario Althéra está formado por dos estrellas de características bien contrastadas:
Althéra A, más masiva y luminosa, con 1,28 masas solares, un radio de 1,32 radios solares y una temperatura superficial de 6.350 K, y Althéra B, más pequeña y fría, con 0,82 masas solares, 0,74 radios solares y 4.950 K. Aunque la luminosidad de Althéra A ( 2,4 veces la del Sol) domina el aporte energético global, la
![img-0.jpeg](img-0.jpeg)

Figure 1 Atardecer en Althera
presencia de Althéra B, con una luminosidad de $0,39 \mathrm{~L} \odot$, actúa como un regulador térmico que suaviza las variaciones extremas de temperatura en la zona habitable, contribuyendo a su estabilidad a largo plazo.

| Parámetro | Althéra A | Althéra B |
| :-- | :-- | :-- |
| Masa | $1,28 \mathrm{M} \odot$ | $0,82 \mathrm{M} \odot$ |
| Radio | $1,32 \mathrm{RO}$ | $0,74 \mathrm{RO}$ |
| Luminosidad | $2,4 \mathrm{~L} \odot$ | $0,39 \mathrm{~L} \odot$ |
| Temperatura superficial | 6.350 K | 4.950 K |

La diferencia de luminosidad significa que, aunque Althéra A domina el aporte energético, Althéra B contribuye de manera significativa a la estabilidad térmica del sistema, suavizando las variaciones extremas en la zona habitable.

chunk: 10 ----------------
chunk.reason_of_chunk=<ChunkReason.section: 'section'>
chunk.summary='Análisis de metalicidad revela que las estrellas de Althéra poseen composiciones similares, sugiriendo un origen común hace aproximadamente 4900 millones de años, edad cercana a la del Sol.'
chunk.sections_found=['3.3 Ciclo de vida y edad estimada del sistema binario']
Text:


### 3.3 Ciclo de vida y edad estimada del sistema binario

Estudios de metalicidad realizados con el ELT indican que ambas estrellas tienen composiciones químicas similares, con una proporción de elementos pesados algo mayor que la del Sol ( $[\mathrm{Fe} / \mathrm{H}] \approx+0,08$ ). Esto sugiere que se formaron a partir de la misma nube molecular hace aproximadamente $\mathbf{4 . 9 0 0}$ millones de años, una edad comparable a la del Sistema Solar.

In [16]:
corpus_llm = []

for chunk in chunks.chunks:
    doc_i = Document(
        page_content=chunk.text,
        metadata={"source": "Althera",
                  "sections_found": chunk.sections_found,
                  "reason_of_chunk": chunk.reason_of_chunk.value}
    )
    corpus_llm.append(doc_i)

In [17]:
utils.save_documents(
    corpus_llm,
    path_output /  "llmbased" / "Althera.json"
)
